In [41]:
import requests
import pandas as pd
import time
import datetime
import random
import sys, os
this_path = '/home/ibi/Documents/GitHub/mas291-project/'
sys.path.append(this_path)
os.chdir(this_path)
print(os.getcwd())

/home/ibi/Documents/GitHub/mas291-project


In [3]:
def dynamic_delay(minimum=1.0, maximum=3.0):
    return random.uniform(minimum, maximum)

def exponential_backoff_retry(request_func, max_retries=6):
    base_wait = 2
    for attempt in range(max_retries):
        try:
            response = request_func()
            if response.ok:
                time.sleep(dynamic_delay(1, 2))
                return response
        except Exception as e:
            print(f"Request failed with exception {e}. Attempt {attempt + 1}/{max_retries}.")
        
        time.sleep(base_wait * (2 ** attempt) + dynamic_delay())

    return None

In [15]:
def scrape_properties(city_slug, city_name, status=["for_sale", "ready_to_build"]):
    url = 'https://www.realtor.com/api/v1/rdc_search_srp?client_id=rdc-search-for-sale-search&schema=vesta'
    headers = {"content-type": "application/json"}
    limit = 200
    all_properties = []
    total_checked = 0
    earliest_date = datetime.datetime.strptime('1995-01-01', "%Y-%m-%d")
    current_date = datetime.datetime.now()

    def send_request(body):
        return requests.post(url, headers=headers, json=body)

    if "sold" in status:
        while current_date > earliest_date:
            start_date = current_date - datetime.timedelta(days=365)
            last_total = None

            while True:
                query = {
                    "sold_date": {"min": start_date.strftime("%Y-%m-%d"), "max": current_date.strftime("%Y-%m-%d")},
                    "status": status,
                    "search_location": {"location": city_name},
                    "type": ["single_family"]
                }

                graphql_query = """
                    query ConsumerSearchQuery(
                        $query: HomeSearchCriteria!
                        $limit: Int
                        $offset: Int
                        $search_promotion: SearchPromotionInput
                        $sort: [SearchAPISort]
                        $sort_type: SearchSortType
                        $client_data: JSON
                        $bucket: SearchAPIBucket
                        ) {
                        home_search: home_search(
                            query: $query
                            sort: $sort
                            limit: $limit
                            offset: $offset
                            sort_type: $sort_type
                            client_data: $client_data
                            bucket: $bucket
                            search_promotion: $search_promotion
                        ) {
                            count
                            total
                            search_promotion {
                            names
                            slots
                            promoted_properties {
                                id
                                from_other_page
                            }
                            }
                            mortgage_params {
                            interest_rate
                            }
                            properties: results {
                            property_id
                            list_price
                            search_promotions {
                                name
                                asset_id
                            }
                            primary_photo(https: true) {
                                href
                            }
                            rent_to_own {
                                right_to_purchase
                                rent
                            }
                            listing_id
                            matterport
                            virtual_tours {
                                href
                                type
                            }
                            status
                            products {
                                products
                                brand_name
                            }
                            source {
                                id
                                type
                                spec_id
                                plan_id
                                agents {
                                office_name
                                }
                            }
                            lead_attributes {
                                show_contact_an_agent
                                opcity_lead_attributes {
                                cashback_enabled
                                flip_the_market_enabled
                                }
                                lead_type
                                ready_connect_mortgage {
                                show_contact_a_lender
                                show_veterans_united
                                }
                            }
                            community {
                                description {
                                name
                                }
                                property_id
                                permalink
                                advertisers {
                                office {
                                    hours
                                    phones {
                                    type
                                    number
                                    primary
                                    trackable
                                    }
                                }
                                }
                                promotions {
                                description
                                href
                                headline
                                }
                            }
                            permalink
                            price_reduced_amount
                            description {
                                name
                                beds
                                baths_consolidated
                                sqft
                                lot_sqft
                                baths_max
                                baths_min
                                beds_min
                                beds_max
                                sqft_min
                                sqft_max
                                type
                                sub_type
                                sold_price
                                sold_date
                            }
                            location {
                                street_view_url
                                address {
                                line
                                postal_code
                                state
                                state_code
                                city
                                coordinate {
                                    lat
                                    lon
                                }
                                }
                                county {
                                name
                                fips_code
                                }
                            }
                            open_houses {
                                start_date
                                end_date
                            }
                            branding {
                                type
                                name
                                photo
                            }
                            flags {
                                is_coming_soon
                                is_new_listing(days: 14)
                                is_price_reduced(days: 30)
                                is_foreclosure
                                is_new_construction
                                is_pending
                                is_contingent
                            }
                            list_date
                            photos(limit: 2, https: true) {
                                href
                            }
                            advertisers {
                                type
                                builder {
                                name
                                href
                                logo
                                }
                            }
                            }
                        }
                        commute_polygon: get_commute_polygon(query: $query) {
                            areas {
                            id
                            breakpoints {
                                width
                                height
                                zoom
                            }
                            radius
                            center {
                                lat
                                lng
                            }
                            }
                            boundary
                        }
                        }
                """

                body = {
                    "query": graphql_query,
                    "variables": {
                        "geoSupportedSlug": city_slug,
                        "query": query,
                        "client_data": {"device_data": {"device_type": "desktop"}},
                        "limit": limit,
                        "offset": 0,
                        "sort": [{"field": "sold_date", "direction": "desc"}],
                        "search_promotion": {"names": ["CITY"], "slots": [], "promoted_properties": []}
                    },
                    "isClient": True,
                    "visitor_id": "7ffa9c49-550f-4c23-aa1b-e93786671450"
                }

                response = exponential_backoff_retry(lambda: send_request(body))
                if response is None or 'data' not in response.json() or 'home_search' not in response.json()['data'] or response.json()['data']['home_search'] is None:
                    print("Error: Invalid response structure or missing data.")
                    print(response.json() if response else "No response")
                    break

                data = response.json()
                api_total = data['data']['home_search']['total']
                print(f"Total properties in {city_name} sold between {start_date.strftime('%Y-%m-%d')} and {current_date.strftime('%Y-%m-%d')}: {api_total}")

                if api_total > 10000:
                    start_date += datetime.timedelta(days=30)
                    time.sleep(dynamic_delay(5, 10))
                else:
                    if last_total is None or api_total > last_total:
                        last_total = api_total
                        start_date -= datetime.timedelta(days=1)
                        time.sleep(dynamic_delay(5, 10))
                    else:
                        break
                    
            offset = 0
            while offset < api_total:
                body['variables']['offset'] = offset
                response = exponential_backoff_retry(lambda: send_request(body))
                if response is None:
                    print("Failed to fetch data after retries.")
                    break
                data = response.json()
                current_batch = data['data']['home_search']['properties']
                all_properties.extend(current_batch)
                total_checked += len(current_batch)
                print(f"Fetched {len(current_batch)} properties this batch. Total checked: {total_checked}.")
                offset += limit
                time.sleep(dynamic_delay(2, 3))

            current_date = start_date - datetime.timedelta(days=1)
            time.sleep(dynamic_delay(5, 10))

    else:
        offset = 0
        api_total = None
        first_batch = True

        query = {
            "status": status,
            "search_location": {
                "location": city_name
            },
            "type": ["single_family"]
        }

        graphql_query = """
            query ConsumerSearchQuery(
                $query: HomeSearchCriteria!
                $limit: Int
                $offset: Int
                $search_promotion: SearchPromotionInput
                $sort: [SearchAPISort]
                $sort_type: SearchSortType
                $client_data: JSON
                $bucket: SearchAPIBucket
                ) {
                home_search: home_search(
                    query: $query
                    sort: $sort
                    limit: $limit
                    offset: $offset
                    sort_type: $sort_type
                    client_data: $client_data
                    bucket: $bucket
                    search_promotion: $search_promotion
                ) {
                    count
                    total
                    search_promotion {
                    names
                    slots
                    promoted_properties {
                        id
                        from_other_page
                    }
                    }
                    mortgage_params {
                    interest_rate
                    }
                    properties: results {
                    property_id
                    list_price
                    search_promotions {
                        name
                        asset_id
                    }
                    primary_photo(https: true) {
                        href
                    }
                    rent_to_own {
                        right_to_purchase
                        rent
                    }
                    listing_id
                    matterport
                    virtual_tours {
                        href
                        type
                    }
                    status
                    products {
                        products
                        brand_name
                    }
                    source {
                        id
                        type
                        spec_id
                        plan_id
                        agents {
                        office_name
                        }
                    }
                    lead_attributes {
                        show_contact_an_agent
                        opcity_lead_attributes {
                        cashback_enabled
                        flip_the_market_enabled
                        }
                        lead_type
                        ready_connect_mortgage {
                        show_contact_a_lender
                        show_veterans_united
                        }
                    }
                    community {
                        description {
                        name
                        }
                        property_id
                        permalink
                        advertisers {
                        office {
                            hours
                            phones {
                            type
                            number
                            primary
                            trackable
                            }
                        }
                        }
                        promotions {
                        description
                        href
                        headline
                        }
                    }
                    permalink
                    price_reduced_amount
                    description {
                        name
                        beds
                        baths_consolidated
                        sqft
                        lot_sqft
                        baths_max
                        baths_min
                        beds_min
                        beds_max
                        sqft_min
                        sqft_max
                        type
                        sub_type
                        sold_price
                        sold_date
                    }
                    location {
                        street_view_url
                        address {
                        line
                        postal_code
                        state
                        state_code
                        city
                        coordinate {
                            lat
                            lon
                        }
                        }
                        county {
                        name
                        fips_code
                        }
                    }
                    open_houses {
                        start_date
                        end_date
                    }
                    branding {
                        type
                        name
                        photo
                    }
                    flags {
                        is_coming_soon
                        is_new_listing(days: 14)
                        is_price_reduced(days: 30)
                        is_foreclosure
                        is_new_construction
                        is_pending
                        is_contingent
                    }
                    list_date
                    photos(limit: 2, https: true) {
                        href
                    }
                    advertisers {
                        type
                        builder {
                        name
                        href
                        logo
                        }
                    }
                    }
                }
                commute_polygon: get_commute_polygon(query: $query) {
                    areas {
                    id
                    breakpoints {
                        width
                        height
                        zoom
                    }
                    radius
                    center {
                        lat
                        lng
                    }
                    }
                    boundary
                }
                }
        """

        while api_total is None or offset < api_total:
            body = {
                "query": graphql_query,
                "variables": {
                    "geoSupportedSlug": city_slug,
                    "query": query,
                    "client_data": {"device_data": {"device_type": "desktop"}},
                    "limit": limit,
                    "offset": offset,
                    "sort_type": "relevant",
                    "search_promotion": {"names": ["CITY"], "slots": [], "promoted_properties": []}
                },
                "isClient": True,
                "visitor_id": "7ffa9c49-550f-4c23-aa1b-e93786671450"
            }

            response = exponential_backoff_retry(lambda: send_request(body))
            if response is None:
                print("Failed to fetch data after retries.")
                break

            data = response.json()

            if api_total is None:
                api_total = data['data']['home_search']['total']
                if first_batch:
                    print(f"Total properties available for sale in {city_name}: {api_total}")
                    first_batch = False
                    
            properties = data['data']['home_search']['properties']
            all_properties.extend(properties)
            total_checked += len(properties)
            print(f"Fetched {len(properties)} selling properties currently. Total available: {api_total}. Total checked: {total_checked}.")
            offset += limit
            time.sleep(dynamic_delay(2, 3))

    return {
        "total_properties_checked": total_checked,
        "properties": all_properties
    }

chicago_data = scrape_properties("Chicago_IL", "Chicago, IL")
print(f"Done fetching {chicago_data['total_properties_checked']} properties selling in Chicago.")
time.sleep(dynamic_delay(50, 60))

chicago_data_sold = scrape_properties("Chicago_IL", "Chicago, IL", ["sold"])
print(f"Done fetching {chicago_data_sold['total_properties_checked']} sold properties in Chicago.")
time.sleep(dynamic_delay(300, 600))

new_york_data = scrape_properties("New-York_NY", "New York, NY")
print(f"Done fetching {new_york_data['total_properties_checked']} properties selling in New York.")
time.sleep(dynamic_delay(50, 60))

new_york_data_sold = scrape_properties("New-York_NY", "New York, NY", ["sold"])
print(f"Done fetching {new_york_data_sold['total_properties_checked']} sold properties in New York.")

Total properties available for sale in Chicago, IL: 3092
Fetched 200 selling properties currently. Total available: 3092. Total checked: 200.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 400.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 600.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 800.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 1000.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 1200.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 1400.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 1600.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 1800.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 2000.
Fetched 200 selling properties currently. Total available: 3092. Total checked: 2200.
F

In [26]:
chicago_json = chicago_data['properties']
chicago_selling = pd.DataFrame(chicago_json)
chicago_selling.head()

,property_id,list_price,search_promotions,primary_photo,rent_to_own,listing_id,matterport,virtual_tours,status,products,...,permalink,price_reduced_amount,description,location,open_houses,branding,flags,list_date,photos,advertisers
0,7109976785,215000.0,None,{'href': 'https://ap.rdcpix.com/696e87b0527eda...,None,2966364640,False,None,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,9906-S-Seeley-Ave_Chicago_IL_60643_M71099-76785,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'P.R.S. Associates...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-04-18T17:33:51.000000Z,[{'href': 'https://ap.rdcpix.com/696e87b0527ed...,"[{'type': 'seller', 'builder': None}]"
1,9462841010,340000.0,None,{'href': 'https://ap.rdcpix.com/82ea120aaba299...,None,2968097446,False,[{'href': 'https://kuula.co/share/5vfHt/collec...,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,650-N-Ridgeway-Ave_Chicago_IL_60624_M94628-41010,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Inherent Homes LL...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-06-03T19:03:15.000000Z,[{'href': 'https://ap.rdcpix.com/82ea120aaba29...,"[{'type': 'seller', 'builder': None}]"
2,7365488014,134900.0,None,{'href': 'https://ap.rdcpix.com/43237d8f85b1ea...,None,2969248556,False,None,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,5519-W-Gladys-Ave_Chicago_IL_60644_M73654-88014,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Parkvue Realty Co...","{'is_coming_soon': None, 'is_new_listing': Tru...",2024-07-05T21:12:51.000000Z,[{'href': 'https://ap.rdcpix.com/43237d8f85b1e...,"[{'type': 'seller', 'builder': None}]"
3,7477995453,69000.0,None,{'href': 'https://ap.rdcpix.com/b929df4ff05fb0...,None,2969234513,False,None,for_sale,"{'products': ['core.agent', 'core.broker', 'co...",...,5237-S-Morgan-St_Chicago_IL_60609_M74779-95453,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'JENNINGS REALTY,I...","{'is_coming_soon': None, 'is_new_listing': Tru...",2024-07-05T14:48:50.000000Z,[{'href': 'https://ap.rdcpix.com/b929df4ff05fb...,"[{'type': 'seller', 'builder': None}]"
4,7913610721,479900.0,None,{'href': 'https://ap.rdcpix.com/c70a366ad55262...,None,2969241582,False,None,for_sale,"{'products': ['core.agent', 'core.broker', 'co...",...,2633-W-Howard-St_Chicago_IL_60645_M79136-10721,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Sky High Real Est...","{'is_coming_soon': None, 'is_new_listing': Tru...",2024-07-05T17:50:39.000000Z,[{'href': 'https://ap.rdcpix.com/c70a366ad5526...,"[{'type': 'seller', 'builder': None}]"


In [27]:
chicago_sold_json = chicago_data_sold['properties']
chicago_sold = pd.DataFrame(chicago_sold_json)
chicago_sold.head()

,property_id,list_price,search_promotions,primary_photo,rent_to_own,listing_id,matterport,virtual_tours,status,products,...,permalink,price_reduced_amount,description,location,open_houses,branding,flags,list_date,photos,advertisers
0,9016962474,1199000.0,None,{'href': 'https://ap.rdcpix.com/33a21ffbb65e7d...,None,2969239196,False,None,sold,"{'products': ['core.agent', 'core.broker', 'co...",...,3052-N-Spaulding-Ave_Chicago_IL_60618_M90169-6...,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Jameson Sotheby's...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-07-05T16:46:20.000000Z,[{'href': 'https://ap.rdcpix.com/33a21ffbb65e7...,"[{'type': 'seller', 'builder': None}]"
1,8304938977,365000.0,None,{'href': 'https://ap.rdcpix.com/ca916ac9fbebe3...,None,2968325620,False,None,sold,"{'products': ['core.agent', 'co_broke'], 'bran...",...,5248-S-Kostner-Ave_Chicago_IL_60632_M83049-38977,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'XR Realty', 'phot...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-06-09T14:51:48.000000Z,[{'href': 'https://ap.rdcpix.com/ca916ac9fbebe...,"[{'type': 'seller', 'builder': None}]"
2,7638203850,250000.0,None,{'href': 'https://ap.rdcpix.com/6b035feede013a...,None,2967945980,False,[{'href': 'https://tours.databasedads.com/4605...,sold,"{'products': ['core.agent', 'core.broker', 'co...",...,10330-S-Whipple-St_Chicago_IL_60655_M76382-03850,NaN,"{'name': None, 'beds': 2, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Berkshire Hathawa...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-05-30T13:32:07.000000Z,[{'href': 'https://ap.rdcpix.com/6b035feede013...,"[{'type': 'seller', 'builder': None}]"
3,8620945945,215000.0,None,{'href': 'https://ap.rdcpix.com/105c64da6141b6...,None,2963388318,False,None,sold,"{'products': ['core.agent', 'co_broke'], 'bran...",...,9137-S-Halsted-St_Chicago_IL_60620_M86209-45945,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Abs Property Cons...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-01-22T22:48:51.000000Z,[{'href': 'https://ap.rdcpix.com/105c64da6141b...,"[{'type': 'seller', 'builder': None}]"
4,8688450280,99900.0,None,{'href': 'https://ap.rdcpix.com/11b4d8082e779a...,None,2964640416,False,None,sold,"{'products': ['core.agent', 'co_broke'], 'bran...",...,11730-S-State-St_Chicago_IL_60628_M86884-50280,NaN,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Chicago Real Esta...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-03-02T04:33:46.000000Z,[{'href': 'https://ap.rdcpix.com/11b4d8082e779...,"[{'type': 'seller', 'builder': None}]"


In [28]:
new_york_json = new_york_data['properties']
new_york_selling = pd.DataFrame(new_york_json)
new_york_selling.head()

,property_id,list_price,search_promotions,primary_photo,rent_to_own,listing_id,matterport,virtual_tours,status,products,...,permalink,price_reduced_amount,description,location,open_houses,branding,flags,list_date,photos,advertisers
0,3243883955,260000,None,{'href': 'https://ap.rdcpix.com/777a0a838fb235...,None,2962223385,False,None,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,620-Sinclair-Ave_Staten-Island_NY_10312_M32438...,NaN,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Sowae Corp', 'pho...","{'is_coming_soon': None, 'is_new_listing': Fal...",2023-12-04T23:24:59.000000Z,[{'href': 'https://ap.rdcpix.com/777a0a838fb23...,"[{'type': 'seller', 'builder': None}]"
1,3974591407,99000,None,{'href': 'https://ap.rdcpix.com/625c0dbb654a69...,None,2965322798,False,None,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,77-City-Blvd_Staten-Island_NY_10301_M39745-91407,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'NYC SHORT SALES B...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-03-20T18:26:05.000000Z,[{'href': 'https://ap.rdcpix.com/625c0dbb654a6...,"[{'type': 'seller', 'builder': None}]"
2,3536803586,250000,None,{'href': 'https://ap.rdcpix.com/9febd1c13274aa...,None,2961898884,False,None,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,24005-147th-Ave_Rosedale_NY_11422_M35368-03586,NaN,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Sowae Corp', 'pho...","{'is_coming_soon': None, 'is_new_listing': Fal...",2023-11-21T20:27:17.000000Z,[{'href': 'https://ap.rdcpix.com/9febd1c13274a...,"[{'type': 'seller', 'builder': None}]"
3,3204384532,275000,None,{'href': 'https://ap.rdcpix.com/bdbd7e0b94a143...,None,2926324318,False,None,for_sale,"{'products': ['co_broke'], 'brand_name': 'basi...",...,579-E-29th-St_Brooklyn_NY_11210_M32043-84532,NaN,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'ONLY NINE REALTY ...","{'is_coming_soon': None, 'is_new_listing': Fal...",2021-02-24T19:23:01.000000Z,[{'href': 'https://ap.rdcpix.com/bdbd7e0b94a14...,"[{'type': 'seller', 'builder': None}]"
4,3767233034,899000,None,{'href': 'https://ap.rdcpix.com/d345414133f77c...,None,2967745369,False,[{'href': 'https://app.doaudiotours.com/unbran...,for_sale,"{'products': ['core.agent', 'co_broke'], 'bran...",...,182-Benedict-Ave_Staten-Island_NY_10314_M37672...,NaN,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Staten Island Pre...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-05-24T18:45:25.000000Z,[{'href': 'https://ap.rdcpix.com/d345414133f77...,"[{'type': 'seller', 'builder': None}]"


In [29]:
new_york_sold_json = new_york_data_sold['properties']
new_york_sold = pd.DataFrame(new_york_sold_json)
new_york_sold.head()

,property_id,list_price,search_promotions,primary_photo,rent_to_own,listing_id,matterport,virtual_tours,status,products,...,permalink,price_reduced_amount,description,location,open_houses,branding,flags,list_date,photos,advertisers
0,3476436735,549999.0,None,{'href': 'https://ap.rdcpix.com/046e20b2fc7528...,None,2965831609,False,"[{'href': 'https://youtu.be/yyKSXSQyIFc', 'typ...",sold,"{'products': ['core.agent', 'co_broke'], 'bran...",...,274-Dixon-Ave_Staten-Island_NY_10303_M34764-36735,NaN,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Martino Realty Gr...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-04-05T15:12:40.000000Z,[{'href': 'https://ap.rdcpix.com/046e20b2fc752...,"[{'type': 'seller', 'builder': None}]"
1,4136891435,1500000.0,None,{'href': 'https://ap.rdcpix.com/1cc7bfa051a25c...,None,2959454563,False,None,sold,"{'products': ['co_broke'], 'brand_name': 'basi...",...,1273-E-10th-St_Brooklyn_NY_11230_M41368-91435,NaN,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'BRISMAN REALTY', ...","{'is_coming_soon': None, 'is_new_listing': Fal...",2023-09-07T17:45:51.000000Z,[{'href': 'https://ap.rdcpix.com/1cc7bfa051a25...,"[{'type': 'seller', 'builder': None}]"
2,4877937449,559000.0,None,{'href': 'https://ap.rdcpix.com/f069417e235ee9...,None,2959829830,False,None,sold,"{'products': ['core.agent', 'co_broke'], 'bran...",...,132-Elm-St_Staten-Island_NY_10310_M48779-37449,10000.0,"{'name': None, 'beds': 4, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Coldwell Banker A...","{'is_coming_soon': None, 'is_new_listing': Fal...",2023-09-20T15:50:10.000000Z,[{'href': 'https://ap.rdcpix.com/f069417e235ee...,"[{'type': 'seller', 'builder': None}]"
3,4669261316,1099000.0,None,{'href': 'https://ap.rdcpix.com/0752b5f9fbeac8...,None,2958513934,False,None,sold,"{'products': ['core.agent', 'core.broker', 'li...",...,1714-E-29th-St_Brooklyn_NY_11229_M46692-61316,50000.0,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'RE MAX Edge', 'ph...","{'is_coming_soon': None, 'is_new_listing': Fal...",2023-08-09T18:18:27.000000Z,[{'href': 'https://ap.rdcpix.com/0752b5f9fbeac...,"[{'type': 'seller', 'builder': None}]"
4,4114846597,968000.0,None,{'href': 'https://ap.rdcpix.com/d2eadd13e0f856...,None,2965127141,False,None,sold,"{'products': ['core.agent', 'core.broker', 'co...",...,4744-189th-St_Flushing_NY_11358_M41148-46597,30000.0,"{'name': None, 'beds': 3, 'baths_consolidated'...",{'street_view_url': 'https://maps.googleapis.c...,None,"[{'type': 'Office', 'name': 'Maureen Folan R E...","{'is_coming_soon': None, 'is_new_listing': Fal...",2024-03-14T15:14:11.000000Z,[{'href': 'https://ap.rdcpix.com/d2eadd13e0f85...,"[{'type': 'seller', 'builder': None}]"


In [35]:
def extract_data(properties):
    extracted_data = []

    for property in properties:
        id = property.get('property_id', None)
        permalink = property.get('permalink', None)
        post_link = "https://www.realtor.com/realestateandhomes-detail/" + permalink if permalink else None
        price = property.get('list_price', None)

        list_date = property.get('list_date', None)
        list_date = list_date.split('T')[0] if list_date else None

        location = property.get('location', {})
        address_line = location.get('address', {}).get('line', None)
        city = location.get('address', {}).get('city', None)
        state_code = location.get('address', {}).get('state_code', None)
        postal_code = location.get('address', {}).get('postal_code', None)
        address = f"{address_line}, {city}, {state_code} {postal_code}" if all([address_line, city, state_code, postal_code]) else None

        status = property.get('status', None)
        status = status.upper() if status else None

        if status == "SOLD":
            sold_date = property.get('description', {}).get('sold_date', None)
            if list_date and sold_date:
                list_date = datetime.datetime.strptime(list_date, "%Y-%m-%d")
                sold_date = datetime.datetime.strptime(sold_date, "%Y-%m-%d")
                days_until_sold = (sold_date - list_date).days
                sold_date = sold_date.strftime("%Y-%m-%d")
                list_date = list_date.strftime("%Y-%m-%d")
            else:
                days_until_sold = None
        else:
            sold_date = "Not sold yet"
            days_until_sold = "Not sold yet"

        description = property.get('description', {})
        area = description.get('sqft', None)
        bedrooms = description.get('beds', None)
        bathrooms = description.get('baths_consolidated', None)

        coordinate = location.get('address', {}).get('coordinate', None)
        latitude = coordinate['lat'] if coordinate else None
        longitude = coordinate['lon'] if coordinate else None

        extracted_data.append({
            'Data Source': 'https://www.realtor.com/',
            'ID': id,
            'Post link': post_link,
            'List date': list_date,
            'Sold date': sold_date,
            'Days until sold': days_until_sold,
            'Price': price,
            'Address': address,
            'Status': status,
            'Area': area,
            'Bedrooms': bedrooms,
            'Bathrooms': bathrooms,
            'Latitude': latitude,
            'Longitude': longitude
        })

    return extracted_data

batch = [chicago_json, chicago_sold_json, new_york_json, new_york_sold_json]
extracted_batches = [[], [], [], []]

for i, properties in enumerate(batch):
    extracted_batches[i] = extract_data(properties)

In [36]:
chicago_selling_extracted = pd.DataFrame(extracted_batches[0])
chicago_selling_extracted.head()

,Data Source,ID,Post link,List date,Sold date,Days until sold,Price,Address,Status,Area,Bedrooms,Bathrooms,Latitude,Longitude
0,https://www.realtor.com/,7109976785,https://www.realtor.com/realestateandhomes-det...,2024-04-18,Not sold yet,Not sold yet,215000.0,"9906 S Seeley Ave, Chicago, IL 60643",FOR_SALE,1567.0,3,2,41.713489,-87.673732
1,https://www.realtor.com/,9462841010,https://www.realtor.com/realestateandhomes-det...,2024-06-03,Not sold yet,Not sold yet,340000.0,"650 N Ridgeway Ave, Chicago, IL 60624",FOR_SALE,1445.0,3,2,NaN,NaN
2,https://www.realtor.com/,7365488014,https://www.realtor.com/realestateandhomes-det...,2024-07-05,Not sold yet,Not sold yet,134900.0,"5519 W Gladys Ave, Chicago, IL 60644",FOR_SALE,1120.0,3,1,41.875399,-87.762458
3,https://www.realtor.com/,7477995453,https://www.realtor.com/realestateandhomes-det...,2024-07-05,Not sold yet,Not sold yet,69000.0,"5237 S Morgan St, Chicago, IL 60609",FOR_SALE,1006.0,3,1,41.798562,-87.649779
4,https://www.realtor.com/,7913610721,https://www.realtor.com/realestateandhomes-det...,2024-07-05,Not sold yet,Not sold yet,479900.0,"2633 W Howard St, Chicago, IL 60645",FOR_SALE,2280.0,3,2,42.019140,-87.696208


In [37]:
chicago_sold_extracted = pd.DataFrame(extracted_batches[1])
chicago_sold_extracted.head()

,Data Source,ID,Post link,List date,Sold date,Days until sold,Price,Address,Status,Area,Bedrooms,Bathrooms,Latitude,Longitude
0,https://www.realtor.com/,9016962474,https://www.realtor.com/realestateandhomes-det...,2024-07-05,2024-07-05,0.0,1199000.0,"3052 N Spaulding Ave, Chicago, IL 60618",SOLD,3387.0,3.0,3.5,NaN,NaN
1,https://www.realtor.com/,8304938977,https://www.realtor.com/realestateandhomes-det...,2024-06-09,2024-07-05,26.0,365000.0,"5248 S Kostner Ave, Chicago, IL 60632",SOLD,NaN,3.0,3,41.797154,-87.733424
2,https://www.realtor.com/,7638203850,https://www.realtor.com/realestateandhomes-det...,2024-05-30,2024-07-03,34.0,250000.0,"10330 S Whipple St, Chicago, IL 60655",SOLD,926.0,2.0,1,41.705251,-87.698129
3,https://www.realtor.com/,8620945945,https://www.realtor.com/realestateandhomes-det...,2024-01-22,2024-07-03,163.0,215000.0,"9137 S Halsted St, Chicago, IL 60620",SOLD,988.0,3.0,2,41.727680,-87.643001
4,https://www.realtor.com/,8688450280,https://www.realtor.com/realestateandhomes-det...,2024-03-02,2024-07-03,123.0,99900.0,"11730 S State St, Chicago, IL 60628",SOLD,2000.0,4.0,2,41.680657,-87.623118


In [38]:
new_york_selling_extracted = pd.DataFrame(extracted_batches[2])
new_york_selling_extracted.head()

,Data Source,ID,Post link,List date,Sold date,Days until sold,Price,Address,Status,Area,Bedrooms,Bathrooms,Latitude,Longitude
0,https://www.realtor.com/,3243883955,https://www.realtor.com/realestateandhomes-det...,2023-12-04,Not sold yet,Not sold yet,260000,"620 Sinclair Ave, Staten Island, NY 10312",FOR_SALE,2015.0,4,2,40.541781,-74.196109
1,https://www.realtor.com/,3974591407,https://www.realtor.com/realestateandhomes-det...,2024-03-20,Not sold yet,Not sold yet,99000,"77 City Blvd, Staten Island, NY 10301",FOR_SALE,1176.0,3,1,40.628757,-74.104166
2,https://www.realtor.com/,3536803586,https://www.realtor.com/realestateandhomes-det...,2023-11-21,Not sold yet,Not sold yet,250000,"240-05 147 Ave, Rosedale, NY 11422",FOR_SALE,2304.0,4,2,40.657539,-73.743602
3,https://www.realtor.com/,3204384532,https://www.realtor.com/realestateandhomes-det...,2021-02-24,Not sold yet,Not sold yet,275000,"579 E 29th St, Brooklyn, NY 11210",FOR_SALE,NaN,4,1,40.636665,-73.948744
4,https://www.realtor.com/,3767233034,https://www.realtor.com/realestateandhomes-det...,2024-05-24,Not sold yet,Not sold yet,899000,"182 Benedict Ave, Staten Island, NY 10314",FOR_SALE,2352.0,4,5,40.622440,-74.128401


In [39]:
new_york_sold_extracted = pd.DataFrame(extracted_batches[3])
new_york_sold_extracted.head()

,Data Source,ID,Post link,List date,Sold date,Days until sold,Price,Address,Status,Area,Bedrooms,Bathrooms,Latitude,Longitude
0,https://www.realtor.com/,3476436735,https://www.realtor.com/realestateandhomes-det...,2024-04-05,2024-07-03,89.0,549999.0,"274 Dixon Ave, Staten Island, NY 10303",SOLD,1205.0,3.0,1.5,40.628432,-74.151338
1,https://www.realtor.com/,4136891435,https://www.realtor.com/realestateandhomes-det...,2023-09-07,2024-07-03,300.0,1500000.0,"1273 E 10th St, Brooklyn, NY 11230",SOLD,2015.0,4.0,1.5,40.618168,-73.964649
2,https://www.realtor.com/,4877937449,https://www.realtor.com/realestateandhomes-det...,2023-09-20,2024-07-03,287.0,559000.0,"132 Elm St, Staten Island, NY 10310",SOLD,1292.0,4.0,2,40.638510,-74.114339
3,https://www.realtor.com/,4669261316,https://www.realtor.com/realestateandhomes-det...,2023-08-09,2024-07-03,329.0,1099000.0,"1714 E 29th St, Brooklyn, NY 11229",SOLD,1660.0,3.0,2.5,40.609290,-73.944877
4,https://www.realtor.com/,4114846597,https://www.realtor.com/realestateandhomes-det...,2024-03-14,2024-07-03,111.0,968000.0,"47-44 189 St, Flushing, NY 11358",SOLD,NaN,3.0,2,40.749657,-73.790139


In [44]:
chicago_selling_extracted.to_csv('data/crawled/chicago_realtor_selling.csv', index=False)
chicago_sold_extracted.to_csv('data/crawled/chicago_realtor_sold.csv', index=False)
new_york_selling_extracted.to_csv('data/crawled/new_york_realtor_selling.csv', index=False)
new_york_sold_extracted.to_csv('data/crawled/new_york_realtor_sold.csv', index=False)